# Tarea para el Hogar 04

##  1. Big Picture

LightGBM es el algoritmo estado del arte para datasets estructurados.
<br> La Bayesian Optimization es el estado del arte para optimización de hiperparámetros
<br> Las soluciones a las tres competencias de la asignatura contendrán LightGBMs y Bayesian Optimizations
<br> LightGBM ha aumentado en forma no darwiniana sus hiperparámetros en los últimos ocho años; no todos los existentes son útiles.
<br> Es necesario lograr entender cuales son los hiperparámetros relevantes de LightGBM

## 2. Hiperparámetros del LightGBM

Los objetivos de esta tarea son:


*   Aumentar la rentabilidad de la campaña de marketing de retención proactiva de clientes.
*   Generar un mejor modelo optimizando sus hiperparámetros
*   Conceptual : investigar los mas relevantes hiperparámetros de LightGBM
*   Familiarizarse con la Bayesian Optimization, sus largos tiempos de corrida y opciones para reducirlos
*   Familiarizarse con el uso de máquinas virtuales de Google Colab
*   Ver un pipeline completo de optimización de hiperparámetros y puesta en producción

LightGBM cuenta con mas de 60 hiperparámetros, siendo posible utilizar 40 al mismo tiempo, aunque no razonable.
<br> La documentación oficial de los hiperparámetros de LightGBM es  https://lightgbm.readthedocs.io/en/latest/Parameters.html#core-parameters


Se lo alerta sobre que una Optimizacion Bayesiana lleva varias horas de corrida, y usted deberá correr VARIAS optimizaciones para descubrir cuales parámetros conviene optimizar.
<br> A pesar que la próxima clase es recien en viernes 01 de agosto, inicie la tarea con tiempo, aprenda a planificar estratégicamente sus corridas como un@ científ@  de datos.

Es necesario investigar cuales son los hiperparámetros de LightGBM que vale la pena optimizar en una Bayesian Optimization, ya que los realmente utiles son apenas un reducido subconjunto.
<br>Usted deberá investigar cuales son los hiperparámetros mas relevantes de LightGBM, su primer alternativa es preguntándole a su amigo con capacidades especiales ChatGPT o sus endogámicos familiares Claude, DeepSeek, Gemini, Grok, etc
<br> La segunda alternativa es la propia documentación de LightGBM  https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html


Adicionalmente podra buscar información como la que proveen esta diminuta muestra aleatoria de artículos ligeros:
*  https://medium.com/@sarahzouinina/a-deep-dive-into-lightgbm-how-to-choose-and-tune-parameters-7c584945842e
*  https://www.kaggle.com/code/somang1418/tuning-hyperparameters-under-10-minutes-lgbm
*  https://towardsdatascience.com/beginners-guide-to-the-must-know-lightgbm-hyperparameters-a0005a812702/


<br>  La muestra anterior se brinda a modo de ejemplo, usted deberá buscar muuuuchas  fuentes adicionales de información
<br> Tenga presente que LightGBM es el estado del arte en modelado predictivo para datasets estructurado, que son el 90% del trabajo del 95% de los Data Scientists en Argentina.

El desafío de esta tarea es:
* Qué hiperparparámetros conviene optimizar?  Las recomendaciones de los artículos ligeros es siempre sensata?  Sus autores realmente hicieron experimentos o son siemplemente escritores de entretenimiento carente de base científica?
* Elegidos los hiperparámetros, cual es el  <desde, hasta> que se debe utilizar en la Bayesian Optimization ?
* Realmente vale la pena optimizar 10 o 16 hiperparámetros al mismo tiempo ?  No resulta contraproducente una búsqueda en un espacio de tal alta dimensionalidad ?

#### 2.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_01_crudo.csv"
archivo_destino="/content/datasets/competencia_01_crudo.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/competencia_01_crudo.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### Generacion de la clase_ternaria

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

In [ ]:
require( "data.table" )

# leo el dataset
dataset <- fread("../../Competencia 01/competencia_01_crudo.csv" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
    "pos" = .I,
    numero_de_cliente,
    periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 ) ]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]


# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

fwrite( dataset,
    file =  "./competencia_01.csv.gz",
    sep = ","
)

In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

### 2.2 Optimizacion Hiperparámetros

Esta parte se debe correr con el runtime en lenguaje R Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

### 2.2.1 Inicio

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "mié sept 10 00:12:24 2025"

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,652133,34.9,1446652,77.3,1062372,56.8
Vcells,1198279,9.2,8388608,64.0,1928447,14.8


### 2.2.2 Carga de Librerias

Esta parte lleva varios minutos la primera vez en Google Colab

In [3]:
# cargo las librerias que necesito
if(!require("data.table")) install.packages("data.table")
require("data.table")

if(!require("parallel")) install.packages("parallel")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Cargando paquete requerido: data.table

Cargando paquete requerido: parallel

Cargando paquete requerido: R.utils

Cargando paquete requerido: R.oo

Cargando paquete requerido: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Adjuntando el paquete: 'R.oo'


The following object is masked from 'package:R.methodsS3':

    throw


The following objects are masked from 'package:methods':

    getClasses, getMethods


The following objects are masked from 'package:base':

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Adjuntando el paquete: 'R.utils'


The following object is masked from 'package:utils':

    timestamp


The following objects are masked from 'package:base':

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings


Cargando paquete requerido: prime

### 2.2.3 Definicion de Parametros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

In [4]:
PARAM <- list()
PARAM$experimento <- "4940_V2A"
PARAM$semilla_primigenia <- 200003

In [5]:
# training y future
PARAM$train <- c(202101,202102)
PARAM$train_final <- c(202101,202102)
PARAM$future <- c(202104)
PARAM$semilla_kaggle <- 314159 #Semilla para el modelo final que va a Kaggle, primeros números de pi que sean primos.
PARAM$cortes <- seq(6000, 19000, by= 500)

In [6]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM$trainingstrategy$undersampling <- 0.5

In [7]:
# Parametros LightGBM

PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir dart, ni pruebe random_forest
  objective= "binary", #default regression
  metric= "auc", # default "" 
  first_metric_only= FALSE, # default FALSE
  boost_from_average= TRUE, # default TRUE
  feature_pre_filter= FALSE, # default TRUE
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, # Default FALSE
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE, # default FALSE

  num_iterations= 1200, # default 100
  learning_rate= 0.02, # default 0.1
  feature_fraction= 0.5, # default 1
  num_leaves= 750, # default 31
  min_data_in_leaf= 5000 # default 20
)


Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization
- si es un numero entero debe ir makeIntegerParam
- si es un numero real (con decimales) debe ir makeNumericParam

Es muy importante leer cuales son un lower y upper permitidos y además razonables

In [8]:
# Aqui se cargan los bordes de los hiperparametros de la BO
PARAM$hypeparametertuning$hs <- makeParamSet(
  makeNumericParam("min_sum_hessian_in_leaf", lower= 0.001, upper= 0.1),
  makeNumericParam("learning_rate", lower= 0.005, upper= 0.1),
  makeNumericParam("feature_fraction", lower= 0.1, upper= 1.0),
  makeNumericParam("bagging_fraction", lower= 0.0, upper= 1.0),
  makeNumericParam("lambda_l1", lower= 0.0, upper= 10.0),
  makeNumericParam("lambda_l2", lower= 0.0, upper= 10.0),
  makeNumericParam("min_gain_to_split", lower= 0.0, upper= 15.0),
  makeIntegerParam("bagging_freq", lower= 1L, upper= 10L),
  makeIntegerParam("num_iterations", lower= 50L, upper= 3000L),
  makeIntegerParam("max_depth", lower= -1L, upper= 15L),
  makeIntegerParam("num_leaves", lower= 1L, upper= 2048L),
  makeIntegerParam("min_data_in_leaf", lower= 1L, upper= 8000L)
)

A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization: 30 es un valor muy tacaño, pero corre rápido deberia partir de 50, alcanzando los 100 si se dispone de tiempo.

In [9]:
PARAM$hyperparametertuning$iteraciones <- 100 # iteraciones bayesianas

In [10]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

In [11]:
# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )

  return( drealidad )
}

In [12]:
# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}

### 2.2.4  Preprocesamiento

In [13]:
# lectura del dataset
dataset <- fread("./competencia_01.csv.gz", stringsAsFactors= TRUE)

In [14]:
dataset_train <- dataset[foto_mes %in% PARAM$train]

In [15]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [16]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train[, azar := runif(nrow(dataset_train))]
dataset_train[, training := 0L]

dataset_train[
  foto_mes %in%  PARAM$train &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [17]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [18]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train[training == 1L, clase01],
  free_raw_data= FALSE
)

nrow(dtrain)
ncol(dtrain)

[1] 163737

[1] 154

2.2.5 Configuracion Bayesian Optimization

In [19]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la AUC en cross validation del modelo entrenado

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelocv <- lgb.cv(
    data= dtrain,
    nfold= PARAM$hyperparametertuning$xval_folds,
    stratified= TRUE,
    param= param_completo
  )

  # obtengo la ganancia
  AUC <- modelocv$best_score

  # hago espacio en la memoria
  rm(modelocv)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

In [20]:
# Aqui comienza la configuracion de la Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_AUC_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hypeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


2.2.6 Corrida Bayesian Optimization

In [21]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Warning message in mboContinue(kbayesiana):
"Tuning ended with term.iter. No need to continue. Simply returning stored result."


In [ ]:
b_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

[1] "min_sum_hessian_in_leaf" "learning_rate"          
 [3] "feature_fraction"        "bagging_fraction"       
 [5] "lambda_l1"               "lambda_l2"              
 [7] "min_gain_to_split"       "bagging_freq"           
 [9] "num_iterations"          "max_depth"              
[11] "num_leaves"              "min_data_in_leaf"       
[13] "y"                       "dob"                    
[15] "eol"                     "error.message"          
[17] "exec.time"               "ei"                     
[19] "error.model"             "train.time"             
[21] "prop.type"               "propose.time"           
[23] "se"                      "mean"

In [23]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]


PARAM$out$lgbm$y <- tb_bayesiana[1, y]


In [24]:
write_yaml( PARAM, file="PARAM.yml")

In [25]:
print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

   min_sum_hessian_in_leaf learning_rate feature_fraction bagging_fraction
                     <num>         <num>            <num>            <num>
1:              0.09507082    0.03092131        0.3411909         0.480642
   lambda_l1 lambda_l2 min_gain_to_split bagging_freq num_iterations max_depth
       <num>     <num>             <num>        <int>          <int>     <int>
1: 0.5977668  0.554335        0.03946446            7           2044        -1
   num_leaves min_data_in_leaf
        <int>            <int>
1:         83               29
[1] 0.9443793


## 2.3  Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la  optimización bayesiana

In [26]:
# clase01
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

In [27]:
dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

clase_ternaria,N
<fct>,<int>
CONTINUA,320372
BAJA+2,1857
BAJA+1,1453


In [28]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

#### Final Training Hyperparameters

In [29]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 200003

$max_depth
[1] -1

$min_gain_to_split
[1] 0.03946446

$min_sum_hessian_in_leaf
[1] 0.09507082

$lambda_l1
[1] 0.5977668

$lambda_l2
[1] 0.554335

$max_bin
[1] 31

$bagging_fraction
[1] 0.480642

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 2044

$learning_rate
[1] 0.03092131

$feature_fraction
[1] 0.3411909

$num_leaves
[1] 83

$min_data_in_leaf
[1] 29

$bagging_freq
[1] 7

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria y mucho menos cross validation.

In [30]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

In [31]:
# entreno LightGBM

modelo_final <- lgb.train(
  data= dtrain_final,
  param= param_normalizado
)

In [32]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(modelo_final))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

In [33]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...
lgb.save(modelo_final, "modelo.txt" )

### Scoring

Aplico el modelo final a los datos del futuro

In [34]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

In [35]:
# inicilizo el dataset  drealidad
drealidad <- realidad_inicializar( dfuture, PARAM)

#### Tabla Prediccion

In [36]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

In [37]:
PARAM$cortes

[1]  6000  6500  7000  7500  8000  8500  9000  9500 10000 10500 11000 11500
[13] 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 17500
[25] 18000 18500 19000

In [38]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  res <- realidad_evaluar( drealidad, tb_prediccion)

  options(scipen = 999)
  cat( "Envios=", envios, "\t",
    " TOTAL=", res$total,
    "  Public=", res$public,
    " Private=", res$private,
    "\n",
    sep= ""
  )

}

Warning message in dir.create("kaggle"):
"'kaggle' already exists"


Envios=6000	 TOTAL=305600000  Public=276000000 Private=318285714
Envios=6500	 TOTAL=314000000  Public=288200000 Private=325057143
Envios=7000	 TOTAL=323200000  Public=289066667 Private=337828571
Envios=7500	 TOTAL=330800000  Public=291933333 Private=347457143
Envios=8000	 TOTAL=334400000  Public=300400000 Private=348971429
Envios=8500	 TOTAL=337200000  Public=299333333 Private=353428571
Envios=9000	 TOTAL=335200000  Public=306000000 Private=347714286
Envios=9500	 TOTAL=335600000  Public=307333333 Private=347714286
Envios=10000	 TOTAL=340000000  Public=313800000 Private=351228571
Envios=10500	 TOTAL=342800000  Public=322066667 Private=351685714
Envios=11000	 TOTAL=342400000  Public=315933333 Private=353742857
Envios=11500	 TOTAL=339600000  Public=308800000 Private=352800000
Envios=12000	 TOTAL=333600000  Public=304133333 Private=346228571
Envios=12500	 TOTAL=331600000  Public=307066667 Private=342114286
Envios=13000	 TOTAL=329600000  Public=304800000 Private=340228571
Envios=13500	 TOTA

In [39]:
write_yaml( PARAM, file="PARAM.yml")

In [40]:

format(Sys.time(), "%a %b %d %X %Y")

[1] "mié sept 10 00:15:26 2025"

Finalmente usted deberá cargar el resultado de su corrida en la Google Sheet Colaborativa,  hoja **TareaHogar04**
<br> Siéntase libre de agregar las columnas que hagan falta a la planilla

In [2]:
format(Sys.time(), "%a %b %d %X %Y")

# cargo las librerias que necesito
if(!require("data.table")) install.packages("data.table")
require("data.table")

if(!require("parallel")) install.packages("parallel")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

# -----------------------------
# PARAMETROS BASE (se mantuvo igual)
# -----------------------------
PARAM <- list()
PARAM$experimento <- "4940_V2"

# training y future
PARAM$train <- c(202101,202102)
PARAM$train_final <- c(202101,202102)
PARAM$future <- c(202104)

# Semilla para split público/privado de la realidad
PARAM$semilla_kaggle <- 314159 # primeros primos de pi

# cortes de envío
PARAM$cortes <- seq(6000, 19000, by= 500)

# undersampling de CONTINUA (se mantiene igual)
PARAM$trainingstrategy$undersampling <- 0.5

# Folds para CV
PARAM$hyperparametertuning$xval_folds <- 5

# Parametros fijos del LightGBM que se pisan con la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= NA,  # <- se setea por semilla más abajo

  max_depth= -1L,
  min_gain_to_split= 0,
  min_sum_hessian_in_leaf= 0.001,
  lambda_l1= 0.0,
  lambda_l2= 0.0,
  max_bin= 31L,

  bagging_fraction= 1.0,
  pos_bagging_fraction= 1.0,
  neg_bagging_fraction= 1.0,
  is_unbalance= FALSE,
  scale_pos_weight= 1.0,

  drop_rate= 0.1,
  max_drop= 50,
  skip_drop= 0.5,

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)

# Bordes de los hiperparametros de la BO (se mantiene igual)
PARAM$hypeparametertuning$hs <- makeParamSet(
  makeNumericParam("min_sum_hessian_in_leaf", lower= 0.001, upper= 0.1),
  makeNumericParam("learning_rate", lower= 0.001, upper= 0.3),
  makeNumericParam("feature_fraction", lower= 0.1, upper= 1.0),
  makeNumericParam("bagging_fraction", lower= 0.0, upper= 1.0),
  makeNumericParam("lambda_l1", lower= 0.0, upper= 100.0),
  makeNumericParam("lambda_l2", lower= 0.0, upper= 100.0),
  makeNumericParam("min_gain_to_split", lower= 0.0, upper= 15.0),
  makeIntegerParam("bagging_freq", lower= 1L, upper= 10L),
  makeIntegerParam("num_iterations", lower= 50L, upper= 4000L),
  makeIntegerParam("max_depth", lower= -1L, upper= 16L),
  makeIntegerParam("num_leaves", lower= 1L, upper= 2048L),
  makeIntegerParam("min_data_in_leaf", lower= 1L, upper= 8000L)
)

PARAM$hyperparametertuning$iteraciones <- 50

# -----------------------------
# Funciones auxiliares (sin cambios)
# -----------------------------

# particionar agrega una columna llamada fold a un dataset
particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")
  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))
  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]
  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )
  return( drealidad )
}

# evaluo ganancia en los datos de la realidad
realidad_evaluar <- function( prealidad, pprediccion) {
  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]
  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]
  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]
  prealidad[, predicted:=NULL]
  return( res )
}

# -----------------------------
# Lectura del dataset (sin cambios)
# -----------------------------
dataset <- fread("./competencia_01.csv.gz", stringsAsFactors= TRUE)

# -----------------------------
# Semillas a recorrer (5 seeds)
# -----------------------------
SEEDS <- c(200003, 300007, 400009, 500009, 600011)

# -----------------------------
# Loop principal por semilla
# -----------------------------
for (seed_actual in SEEDS) {

  cat("\n========================\n")
  cat("Comenzando semilla:", seed_actual, "\n")
  cat("========================\n")

  # seteo semilla en PARAM y en LightGBM
  PARAM$semilla_primigenia <- seed_actual
  PARAM$lgbm$param_fijos$seed <- seed_actual

  # --------- Preprocesamiento + undersampling (sin cambios de lógica) ---------
  dataset_train <- dataset[foto_mes %in% PARAM$train]

  # clase binaria
  dataset_train[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)]

  # undersampling controlado por semilla
  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
  dataset_train[, azar := runif(nrow(dataset_train))]
  dataset_train[, training := 0L]
  dataset_train[
    foto_mes %in%  PARAM$train &
      (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
    training := 1L
  ]

  # campos a usar
  campos_buenos <- setdiff(
    colnames(dataset_train),
    c("clase_ternaria", "clase01", "azar", "training")
  )

  # dataset para LightGBM
  dtrain <- lgb.Dataset(
    data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
    label= dataset_train[training == 1L, clase01],
    free_raw_data= FALSE
  )

  nrow(dtrain); ncol(dtrain)

  # --------- Función objetivo para la BO (sin cambios) ---------
  EstimarGanancia_AUC_lightgbm <- function(x) {
    param_completo <- modifyList(PARAM$lgbm$param_fijos, x)
    modelocv <- lgb.cv(
      data= dtrain,
      nfold= PARAM$hyperparametertuning$xval_folds,
      stratified= TRUE,
      param= param_completo
    )
    AUC <- modelocv$best_score
    rm(modelocv); gc(full= TRUE, verbose= FALSE)
    message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)
    return(AUC)
  }

  # --------- Configuración BO (sin cambios de lógica) ---------
  kbayesiana <- paste0("bayesiana_", seed_actual, ".RDATA")
  funcion_optimizar <- EstimarGanancia_AUC_lightgbm

  configureMlr(show.learner.output= FALSE)

  obj.fun <- makeSingleObjectiveFunction(
    fn= funcion_optimizar,
    minimize= FALSE,
    noisy= TRUE,
    par.set= PARAM$hypeparametertuning$hs,
    has.simple.signature= FALSE
  )

  ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= kbayesiana
  )

  ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hyperparametertuning$iteraciones
  )

  ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

  surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace= TRUE)
  )

  if (!file.exists(kbayesiana)) {
    bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
  } else {
    bayesiana_salida <- mboContinue(kbayesiana)
  }

  tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
  tb_bayesiana[, iter := .I]
  setorder(tb_bayesiana, -y)

  # log de la BO por semilla
  fwrite(tb_bayesiana, file= paste0("BO_log_", seed_actual, ".txt"), sep= "\t")

  # mejores hiperparametros
  PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
    1,
    setdiff(colnames(tb_bayesiana),
      c("y","dob","eol","error.message","exec.time","ei","error.model",
        "train.time","prop.type","propose.time","se","mean","iter")),
    with= FALSE
  ]
  PARAM$out$lgbm$y <- tb_bayesiana[1, y]

  # guardo PARAM por semilla
  write_yaml(PARAM, file= paste0("PARAM_", seed_actual, ".yml"))
  print(PARAM$out$lgbm$mejores_hiperparametros); print(PARAM$out$lgbm$y)

  # --------- Entrenamiento final por semilla (sin cambios de lógica) ---------
  dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]
  dataset_train_final <- dataset[foto_mes %in% PARAM$train_final]

  dtrain_final <- lgb.Dataset(
    data= data.matrix(dataset_train_final[, campos_buenos, with= FALSE]),
    label= dataset_train_final[, clase01]
  )

  param_final <- modifyList(PARAM$lgbm$param_fijos,
    PARAM$out$lgbm$mejores_hiperparametros)

  # normalización por undersampling (igual que original)
  param_normalizado <- copy(param_final)
  param_normalizado$min_data_in_leaf <- round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

  modelo_final <- lgb.train(
    data= dtrain_final,
    param= param_normalizado
  )

  # importancia por semilla
  tb_importancia <- as.data.table(lgb.importance(modelo_final))
  fwrite(tb_importancia,
    file= paste0("impo_", seed_actual, ".txt"),
    sep= "\t"
  )

  # modelo por semilla
  lgb.save(modelo_final, paste0("modelo_", seed_actual, ".txt"))

  # predicción de futuro por semilla
  dfuture <- dataset[foto_mes %in% PARAM$future]
  prediccion <- predict(
    modelo_final,
    data.matrix(dfuture[, campos_buenos, with= FALSE])
  )

  tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
  tb_prediccion[, prob := prediccion ]

  # guardo predicciones por semilla (intermedio)
  fwrite(tb_prediccion,
    file= paste0("prediccion_", seed_actual, ".txt"),
    sep= "\t"
  )

  # Limpieza mínima de objetos pesados de la semilla
  rm(dtrain, dtrain_final, modelo_final, tb_importancia, tb_bayesiana,
     tb_prediccion, prediccion, dfuture, dataset_train, dataset_train_final)
  gc(full= TRUE, verbose= FALSE)
}

format(Sys.time(), "%a %b %d %X %Y")


[1] "jue sept 11 07:30:26 2025"

Cargando paquete requerido: data.table

Cargando paquete requerido: parallel

Cargando paquete requerido: R.utils

Cargando paquete requerido: R.oo

Cargando paquete requerido: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Adjuntando el paquete: 'R.oo'


The following object is masked from 'package:R.methodsS3':

    throw


The following objects are masked from 'package:methods':

    getClasses, getMethods


The following objects are masked from 'package:base':

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Adjuntando el paquete: 'R.utils'


The following object is masked from 'package:utils':

    timestamp


The following objects are masked from 'package:base':

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings


Cargando paquete requerido: prime


Comenzando semilla: 200003 


Computing y column(s) for design. Not provided.

jue sept 11 07:31:56 2025 AUC 0.918485086308789

jue sept 11 07:32:53 2025 AUC 0.913076704789114

jue sept 11 07:33:31 2025 AUC 0.911579285382297

jue sept 11 07:34:24 2025 AUC 0.919053399126218

jue sept 11 07:35:22 2025 AUC 0.905732335593353

jue sept 11 07:35:42 2025 AUC 0.915373446401831

jue sept 11 07:36:48 2025 AUC 0.91738676269621

jue sept 11 07:38:32 2025 AUC 0.914909677879417

jue sept 11 07:39:27 2025 AUC 0.90838115273461

jue sept 11 07:39:45 2025 AUC 0.9141385263733

jue sept 11 07:40:14 2025 AUC 0.5

jue sept 11 07:42:57 2025 AUC 0.922141670568576

jue sept 11 07:43:59 2025 AUC 0.918292353099481

jue sept 11 07:45:25 2025 AUC 0.912710270538976

jue sept 11 07:45:54 2025 AUC 0.910705475846452

jue sept 11 07:46:26 2025 AUC 0.898021012191964

jue sept 11 07:47:09 2025 AUC 0.917528726293717

jue sept 11 07:47:28 2025 AUC 0.91154207353544

jue sept 11 07:48:40 2025 AUC 0.909346783636744

jue sept 11 07:49:49 2025 AUC 0.8980178

   min_sum_hessian_in_leaf learning_rate feature_fraction bagging_fraction
                     <num>         <num>            <num>            <num>
1:              0.08553213    0.08796551        0.5204523         0.821764
   lambda_l1 lambda_l2 min_gain_to_split bagging_freq num_iterations max_depth
       <num>     <num>             <num>        <int>          <int>     <int>
1:  1.096044  14.15436         0.5284354            2           1735        11
   num_leaves min_data_in_leaf
        <int>            <int>
1:       1448              273
[1] 0.9387842

Comenzando semilla: 300007 


...

Computing y column(s) for design. Not provided.

jue sept 11 11:43:55 2025 AUC 0.918246071805895

jue sept 11 11:45:47 2025 AUC 0.916613306654995

jue sept 11 11:46:47 2025 AUC 0.91062254741162

jue sept 11 11:47:47 2025 AUC 0.918713759933487

jue sept 11 11:48:12 2025 AUC 0.914277988006959

jue sept 11 11:48:31 2025 AUC 0.912210151718085

jue sept 11 11:49:15 2025 AUC 0.902588441095566

jue sept 11 11:50:26 2025 AUC 0.916726469170104

jue sept 11 11:51:37 2025 AUC 0.908917025316141

jue sept 11 11:52:05 2025 AUC 0.918144415437191

jue sept 11 11:53:19 2025 AUC 0.91618964410745

jue sept 11 11:54:09 2025 AUC 0.927147747847538

jue sept 11 11:54:49 2025 AUC 0.900362911768693

jue sept 11 11:55:49 2025 AUC 0.888057413869131

jue sept 11 11:56:58 2025 AUC 0.915749920948103

jue sept 11 11:57:14 2025 AUC 0.921660960994939

jue sept 11 11:57:17 2025 AUC 0.896651599725537

jue sept 11 11:58:32 2025 AUC 0.905309680765517

jue sept 11 11:59:03 2025 AUC 0.89878865209995

jue sept 11 11:59:22 20

   min_sum_hessian_in_leaf learning_rate feature_fraction bagging_fraction
                     <num>         <num>            <num>            <num>
1:              0.01418043    0.03094975        0.2102479         0.747232
   lambda_l1 lambda_l2 min_gain_to_split bagging_freq num_iterations max_depth
       <num>     <num>             <num>        <int>          <int>     <int>
1:  1.857101     50.89         0.1150429            4           2255         6
   num_leaves min_data_in_leaf
        <int>            <int>
1:       1763             3775
[1] 0.9343003

Comenzando semilla: 600011 


Computing y column(s) for design. Not provided.

jue sept 11 13:11:01 2025 AUC 0.919550905519687

jue sept 11 13:12:07 2025 AUC 0.907475254318201

jue sept 11 13:12:23 2025 AUC 0.896303972066246

jue sept 11 13:12:43 2025 AUC 0.912524678193477

jue sept 11 13:13:06 2025 AUC 0.900371129517705

jue sept 11 13:13:47 2025 AUC 0.918353597116081

jue sept 11 13:14:55 2025 AUC 0.912426583123023

jue sept 11 13:15:50 2025 AUC 0.910593327352073

jue sept 11 13:16:11 2025 AUC 0.5

jue sept 11 13:16:50 2025 AUC 0.91177036793178

jue sept 11 13:17:47 2025 AUC 0.918204074961026

jue sept 11 13:18:34 2025 AUC 0.914451852953891

jue sept 11 13:20:36 2025 AUC 0.933231784651631

jue sept 11 13:21:23 2025 AUC 0.807238684246853

jue sept 11 13:23:01 2025 AUC 0.911083792170927

jue sept 11 13:23:46 2025 AUC 0.888581251779196

jue sept 11 13:24:14 2025 AUC 0.909032843777845

jue sept 11 13:24:56 2025 AUC 0.912921148526685

jue sept 11 13:25:57 2025 AUC 0.909860606126069

jue sept 11 13:26:23 2025 AUC 0.923

   min_sum_hessian_in_leaf learning_rate feature_fraction bagging_fraction
                     <num>         <num>            <num>            <num>
1:              0.09949277    0.04545869        0.4430241        0.7880072
   lambda_l1 lambda_l2 min_gain_to_split bagging_freq num_iterations max_depth
       <num>     <num>             <num>        <int>          <int>     <int>
1:  1.264718  43.95391        0.05690343            8           1728        14
   num_leaves min_data_in_leaf
        <int>            <int>
1:        253              725
[1] 0.9397581


[1] "jue sept 11 14:41:03 2025"

In [3]:
format(Sys.time(), "%a %b %d %X %Y")

# cargo librerias necesarias para esta parte
if(!require("data.table")) install.packages("data.table")
require("data.table")

if( !require("yaml")) install.packages("yaml")
require("yaml")

# -----------------------------
# Reutilizo configuración necesaria
# -----------------------------
PARAM <- list()
PARAM$experimento <- "4940_V2"

PARAM$train <- c(202101,202102)
PARAM$train_final <- c(202101,202102)
PARAM$future <- c(202104)

PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(6000, 19000, by= 500)

# funciones de particionado y evaluación (sin cambios)
particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")
  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))
  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}
realidad_inicializar <- function( pfuture, pparam) {
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]
  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )
  return( drealidad )
}
realidad_evaluar <- function( prealidad, pprediccion) {
  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]
  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]
  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]
  prealidad[, predicted:=NULL]
  return( res )
}

# -----------------------------
# Semillas a promediar
# -----------------------------
SEEDS <- c(200003, 300007, 400009, 500009, 600011)

# -----------------------------
# Lectura dataset y armado de "realidad"
# -----------------------------
dataset <- fread("./competencia_01.csv.gz", stringsAsFactors= TRUE)
dfuture <- dataset[foto_mes %in% PARAM$future]
drealidad <- realidad_inicializar(dfuture, PARAM)

# -----------------------------
# Cargo predicciones por semilla y armo ensemble por promedio
# -----------------------------
pred_list <- list()
for (seed_actual in SEEDS) {
  pr <- fread(paste0("prediccion_", seed_actual, ".txt"))
  setnames(pr, "prob", paste0("prob_", seed_actual))
  pred_list[[as.character(seed_actual)]] <- pr
}

# merge secuencial por numero_de_cliente y foto_mes
tb_ens <- Reduce(function(x,y) merge(x, y, by=c("numero_de_cliente","foto_mes")), pred_list)

# promedio simple de las 5 probabilidades
tb_ens[, prob := rowMeans(.SD), .SDcols = paste0("prob_", SEEDS)]

# dejo solo las columnas esperadas por el test (sin modificar su forma)
tb_prediccion <- tb_ens[, .(numero_de_cliente, foto_mes, prob)]

# guardo predicción ensemble (intermedio informativo)
fwrite(tb_prediccion, file= "prediccion_ensemble.txt", sep= "\t")

# -----------------------------
# Generación de archivos Kaggle (misma lógica, sin cambios)
# -----------------------------
# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

# carpeta kaggle
dir.create("kaggle", showWarnings = FALSE)

for (envios in PARAM$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

  # NOTA: si prefieres marcar explícitamente que es ensemble, cambia la línea siguiente por:
  # archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_ENS_", envios, ".csv")
  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo final para subir a Kaggle
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # evaluación con la "realidad" (misma forma)
  res <- realidad_evaluar( drealidad, tb_prediccion)

  options(scipen = 999)
  cat( "Envios=", envios, "\t",
    " TOTAL=", res$total,
    "  Public=", res$public,
    " Private=", res$private,
    "\n",
    sep= ""
  )
}

format(Sys.time(), "%a %b %d %X %Y")


[1] "jue sept 11 14:41:03 2025"

Envios=6000	 TOTAL=325600000  Public=280933333 Private=344742857
Envios=6500	 TOTAL=338000000  Public=288933333 Private=359028571
Envios=7000	 TOTAL=348000000  Public=300266667 Private=368457143
Envios=7500	 TOTAL=351600000  Public=308266667 Private=370171429
Envios=8000	 TOTAL=352800000  Public=309933333 Private=371171429
Envios=8500	 TOTAL=352400000  Public=309533333 Private=370771429
Envios=9000	 TOTAL=351200000  Public=313000000 Private=367571429
Envios=9500	 TOTAL=350000000  Public=317000000 Private=364142857
Envios=10000	 TOTAL=349600000  Public=315000000 Private=364428571
Envios=10500	 TOTAL=349200000  Public=315533333 Private=363628571
Envios=11000	 TOTAL=351200000  Public=316133333 Private=366228571
Envios=11500	 TOTAL=349200000  Public=324133333 Private=359942857
Envios=12000	 TOTAL=347200000  Public=320266667 Private=358742857
Envios=12500	 TOTAL=346800000  Public=323533333 Private=356771429
Envios=13000	 TOTAL=346400000  Public=319666667 Private=357857143
Envios=13500	 TOTA

[1] "jue sept 11 14:41:08 2025"